<a href="https://colab.research.google.com/github/RachanaKarekar/NLP-Projects/blob/master/CharacterBased_LanguageModeling_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

# Create the sequence

---



In [0]:
# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text


# save tokens to file, one dialog per line
def save_doc(lines, filename):
  data = '\n'.join(lines)
  file = open(filename, 'w')
  file.write(data)
  file.close()


# load text
raw_text = load_doc("/content/gdrive/My Drive/NLP_Projects/7/rhyme.txt")
print(raw_text)

# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
  # select sequence of tokens
  seq = raw_text[i-length:i+1]
  
  # store
  sequences.append(seq)

print('Total Sequences: %d' % len(sequences))
# save sequences to file
out_filename = "/content/gdrive/My Drive/NLP_Projects/7/char_sequences.txt"
save_doc(sequences, out_filename)

# Train the model

---



In [4]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


# define the model
def define_model(X):
  model = Sequential()
  model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dense(vocab_size, activation='softmax'))
  # compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarize defined model
  model.summary()
  plot_model(model, to_file='model.png', show_shapes=True)
  return model

# load
in_filename = "/content/gdrive/My Drive/NLP_Projects/7/char_sequences.txt" 
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
  # integer encode line
  encoded_seq = [mapping[char] for char in line]
  # store
  sequences.append(encoded_seq)
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)
# define model
model = define_model(X)
# fit model
model.fit(X, y, epochs=100, verbose=2)
# save the model to file
model.save("/content/gdrive/My Drive/NLP_Projects/7/model.h5")
# save the mapping
dump(mapping, open("/content/gdrive/My Drive/NLP_Projects/7/mapping.pkl", 'wb'))

Using TensorFlow backend.


Vocabulary Size: 38
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                34200     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                2888      
Total params: 37,088
Trainable params: 37,088
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
 - 3s - loss: 3.6217 - accuracy: 0.0852
Epoch 2/100
 - 1s - loss: 3.5278 - accuracy: 0.1905
Epoch 3/100
 - 1s - loss: 3.2265 - accuracy: 0.1905
Epoch 4/100
 - 1s - loss: 3.0600 - accuracy: 0.1905
Epoch 5/100
 - 1s - loss: 3.0175 - accuracy: 0.1905
Epoch 6/100
 - 1s - loss: 2.9927 - accuracy: 0.1905
Epoch 7/100
 - 1s - loss: 2.9722 - accuracy: 0.1905
Epoch 8/100
 - 1s - loss: 2.9598 - accuracy: 0.1905
Epoch 9/100
 - 1s - loss: 2.9369 - accuracy: 0.1905
Epoch 10/100
 - 1s - los

# Generate text

---

As we can see, the texts of the rhyme get replicated without any mistakes. However, if a new text is given, in this case "hello world", the model is not able to generate meaningful responses as it has never seen these sequences while training.

In [8]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
  in_text = seed_text
  # generate a fixed number of characters
  for _ in range(n_chars):
    # encode the characters as integers
    encoded = [mapping[char] for char in in_text]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # one hot encode
    encoded = to_categorical(encoded, num_classes=len(mapping))
    # encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
    # predict character
    yhat = model.predict_classes(encoded, verbose=0)
    # reverse map integer to character
    out_char = ''
    for char, index in mapping.items():
      if index == yhat:
        out_char = char
        break
    # append to input
    in_text += out_char
  return in_text


# load the model
model = load_model("/content/gdrive/My Drive/NLP_Projects/7/model.h5")
# load the mapping
mapping = load(open("/content/gdrive/My Drive/NLP_Projects/7/mapping.pkl", 'rb'))
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))  

Sing a song of sixpence, A poc
king was in his counting house
hello worl, hhe in,., Taa inid
